In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import datetime
import logging
import warnings
from pathlib import Path 

import pandas as pd 
import numpy as np 
from tqdm import tqdm

warnings.filterwarnings('ignore')

### 输出`data_to_model.csv`文件, 内容为经特征工程处理后的`train.csv + test.csv`

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')
# 数据集目录
data_path = Path(r'/Users/liuzhi/datasets/tc_金融风控-贷款违约预测')

In [3]:
train = pd.read_csv(f"{data_path}/train.csv")
testA = pd.read_csv(f"{data_path}/testA.csv")
logging.info(f"train shape: {train.shape}")
logging.info(f"testA shape: {testA.shape}")
train.head()

2021-01-14 20:45:38,466 INFO: train shape: (800000, 47)
2021-01-14 20:45:38,466 INFO: testA shape: (200000, 46)


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


### 特征预处理

In [4]:
# Fucntion from EDA
def split_numerical_and_category_features(df):
    r'''拆分数值列、类别列'''
    numerical_features = list(df.select_dtypes(exclude=['object']).columns)
    category_features = list(filter(lambda x: x not in numerical_features, list(df.columns)))
    return numerical_features, category_features

def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [5]:
# 查找出数据中的对象特征和数值特征
label = 'isDefault'

numerical_features, category_featrues = split_numerical_and_category_features(train)
numerical_features.remove(label)

### 缺失值填充
1. 所有缺失值替换为指定值，如 0
    data = data.fillna(0)

2. 用缺失值上面的值替换缺失值
    data = data.fillna(axis=0, method='ffill')

3. 纵向用缺失值下面的值替换缺失值，且设置最多填充2个连续的缺失值
    data = data.fillna(axis=0, method='bfill', limit=2)

In [6]:
# 按平均值填充数值类型特征
train[numerical_features] = train[numerical_features].fillna(train[numerical_features].median())
testA[numerical_features] = testA[numerical_features].fillna(testA[numerical_features].median())

# 按众数填充类别特征
train[category_featrues] = train[category_featrues].fillna(train[category_featrues].mode())
testA[category_featrues] = testA[category_featrues].fillna(testA[category_featrues].mode())

2021-01-14 20:45:40,632 INFO: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-01-14 20:45:40,633 INFO: NumExpr defaulting to 8 threads.


In [7]:
# 查看填充缺失值后 Train 的情况
train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           0
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  0
regionCode                0
dti                       0
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies        0
revolBal                  0
revolUtil                 0
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     0
policyCode                0
n0                        0
n1                        0
n2                        0
n3                  

#### employmentLength

In [8]:
train['employmentLength'].value_counts(dropna=False).sort_index()

1 year        52489
10+ years    262753
2 years       72358
3 years       64152
4 years       47985
5 years       50102
6 years       37254
7 years       35407
8 years       36192
9 years       30272
< 1 year      64237
NaN           46799
Name: employmentLength, dtype: int64

In [9]:
# 对象类型特征转换到数值
for data in [train, testA]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace(to_replace='< 1 year', value='0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [10]:
train['employmentLength'].value_counts(dropna=False).sort_index()

0.0      64237
1.0      52489
2.0      72358
3.0      64152
4.0      47985
5.0      50102
6.0      37254
7.0      35407
8.0      36192
9.0      30272
10.0    262753
NaN      46799
Name: employmentLength, dtype: int64

#### issueDate

In [11]:
# issueDate为时间格式的特征

# 转换为时间格式
for data in [train,testA]:
    data['issueDate'] = pd.to_datetime(data['issueDate'], format=r'%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', r'%Y-%m-%d')
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

#### 类别特征编码处理

In [12]:
# 部分类别特征处理
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', 'applicationType', 'initialListStatus', 'title', 'policyCode']

for f in cate_features:
    print(f, 'Cate Number:', train[f].nunique())

grade Cate Number: 7
subGrade Cate Number: 35
employmentTitle Cate Number: 248683
homeOwnership Cate Number: 6
verificationStatus Cate Number: 3
purpose Cate Number: 14
postCode Cate Number: 932
regionCode Cate Number: 51
applicationType Cate Number: 2
initialListStatus Cate Number: 2
title Cate Number: 39644
policyCode Cate Number: 1


In [13]:
# 1. 类别数目大于2，且不是高维的特征(如<100)
for data in [train, testA]:
    data = pd.get_dummies(data, columns=['grade', 'subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True)

In [14]:
# 2. 类别数目是高维的特征，需要进行转换

for data in [train, testA]:
    for f in ['employmentTitle', 'postCode', 'title']:
        data[f"{f}_cnts"] = data.groupby([f])['id'].transform('count')
        data[f"{f}_rank"] = data.groupby([f])['id'].rank(ascending=False).astype(int)
        del data[f]

### 合并输出

In [16]:
train['sample'] = 'train'
testA['sample'] = 'test'

# 关联train + test 一起做特征处理
data = pd.concat([train, testA], axis=0, ignore_index=True)
data.to_csv(f"data/data_for_model.csv", index=False)
logging.info(f"combine data shape: {data.shape}")

2021-01-14 21:21:11,881 INFO: combine data shape: (1000000, 52)
